In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [1]:
import torch

print(torch.cuda.is_available())

False


In [3]:
!nvidia-smi

Wed Jan 29 12:55:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   34C    P0             73W /  500W |   33277MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

CPU times: user 317 ms, sys: 174 ms, total: 492 ms
Wall time: 981 ms


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/"
default_yaml = "./prompts/one_shot_training_text_prompt.yaml"
conversation = None

In [4]:
images = sorted(utils.load_images(image_dir))

In [5]:
import lib.pages.roi as roi
images = roi.cropAllImages(images, 100.0, 0.4, 
              0.01, None)

>>>> Splitting successfull : The_Lightfoot_Herbarium_05.jpg | number of middle lines found = 5 | number of lines found = 5
>>>> Splitting successfull : The_Lightfoot_Herbarium_06.jpg | number of middle lines found = 4 | number of lines found = 12
>>>> Splitting successfull : The_Lightfoot_Herbarium_07.jpg | number of middle lines found = 2 | number of lines found = 10
>>>> Splitting successfull : The_Lightfoot_Herbarium_08.jpg | number of middle lines found = 4 | number of lines found = 16
>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_09.jpg | number of middle lines found = 0 | number of lines found = 0
	Trying now with thresholding
	Image The_Lightfoot_Herbarium_09.jpg successfully split down the middle
>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_10.jpg | number of middle lines found = 0 | number of lines found = 0
	Trying now with thresholding
	Image The_Lightfoot_Herbarium_10.jpg successfully split down the middle
>>>> Splitting unsuccessful  : The_Lightfoot_Herb

Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box
Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box


>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_05_cropped_2.jpg | number of middle lines found = 0 | number of lines found = 3
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_05_cropped_2.jpg
>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_06_cropped_1.jpg | number of middle lines found = 0 | number of lines found = 3
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_06_cropped_1.jpg


Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box
Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box


>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_06_cropped_2.jpg | number of middle lines found = 0 | number of lines found = 1
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_06_cropped_2.jpg
>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_07_cropped_1.jpg | number of middle lines found = 0 | number of lines found = 3
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_07_cropped_1.jpg


Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box
Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box
Error in boxClipToRectangle: box outside rectangle
Error in pixScanForForeground: invalid box


>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_07_cropped_2.jpg | number of middle lines found = 0 | number of lines found = 1
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_07_cropped_2.jpg
>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_08_cropped_1.jpg | number of middle lines found = 0 | number of lines found = 4
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_08_cropped_1.jpg
>>>> Splitting unsuccessful  : The_Lightfoot_Herbarium_08_cropped_2.jpg | number of middle lines found = 0 | number of lines found = 3
	Trying now with thresholding
Width of image is less than height
Considering this image to already be split
Skipping image: The_Lightfoot_Herbarium_08_cropped_2.jpg


In [6]:
images = images[:1]

In [7]:
model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 130000, batch_size=1, temperature=1, save_path="./outputs/lightfootcat_test/")

/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [00:24<00:00,  4.92s/it]


In [8]:
outputs = model(images, save=False, debug=False, save_file_name="two_images")

Model: qwen_model | Batch Size: 1, Max Tokens: 130000, Temperature: 1
Extracting Text from Images


Processing Batches: 100%|██████████| 1/1 [00:28<00:00, 28.39s/batch]


Converting extracted text into Text Blocks
Organising text into JSON blocks


Processing Families in Dicotyledones: 100%|██████████| 7/7 [00:37<00:00,  5.40s/family]


In [9]:
outputs

{'Dicotyledones': [{'familyName': 'ACERACEAE',
   'species': [{'speciesName': 'Acer campestre L.',
     'folders': [{'description': 'Acer campestre [TA]'}]},
    {'speciesName': 'Acer pseudoplatanus L.',
     'folders': [{'description': 'Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]'},
      {'description': 'Acer Pseudo-Platanus [TA].'}]}]},
  {'familyName': 'AMARANTHACEAE',
   'species': [{'speciesName': 'amaranthus lividus L.',
     'folders': [{'description': 'Amaranthus Blitum [TA]. i. Cites Ray\'s Syn. 1957. ii. "Blite Amaranth. Aug.: It is often found on Dunghills in the neighbourhood of London. I gather\'d this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]'}]}]},
  {'familyName': 'ARALIACEAE',
   'species': [{'speciesName': 'Hedera helix L.',
     'folders': [{'description': 'Hedera Helix [TA].'}]}]},
  {'familyName': 'ARISTOLOCHIACEAE',
   'species': [{'speciesName': 'Asarum europaeum L.',
     'folders': [{'descr

In [1]:
import pandas

In [2]:
out_csv = pandas.read_csv("outputs/lightfootcat/lightfootcat_full.csv")

In [18]:
out_csv.columns

Index(['division', 'familyName', 'speciesName', 'description', 'folders',
       'location', 'sheet', 'folderNumber'],
      dtype='object')

In [29]:
dict(out_csv.description.value_counts())

{'This Plant commonly produced 1 or 2 flowers when wild: Anglesey.': np.int64(27),
 'Euphorbia platyphylllos. Lin: From Harefield, Middlesex': np.int64(6),
 'Euphorbia Paralias Lin: Sandy sea Shores at Breton Ferry in Glamorganshire': np.int64(6),
 'Vicia sativa': np.int64(5),
 'British and foreign': np.int64(4),
 'Vicia lutea': np.int64(3),
 'Trifolium striatum': np.int64(3),
 'Dentaria bulbifera [TA].': np.int64(3),
 'Trifolium maritimum': np.int64(3),
 'Athanasia maritima [G].': np.int64(3),
 'Taller & more branched, bearing 6 flowers, capsules oblongs.': np.int64(3),
 'Vicia sylvatica': np.int64(2),
 'Gnaphalium dioicum [TA].': np.int64(2),
 'Artemisia maritima [G].': np.int64(2),
 'Lobelia urens [G]': np.int64(2),
 'Carex vulpina [TA]. i. "Carex vulpina. The Brizoides should have come next, but that is a very scarce Plant in NorfolkI will thank you for a good Specimen of it" [B].': np.int64(2),
 'Schoenus mariscus [TA]. i. Cites Ray\'s Syn. 426; Moris. Hist. Sect. 8, t. 11, f. 24.

In [32]:
out_csv[out_csv["description"] == "This Plant commonly produced 1 or 2 flowers when wild: Anglesey."]

,division,familyName,speciesName,description,folders,location,sheet,folderNumber
866,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
867,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
870,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
871,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
872,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
873,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
874,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
875,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
876,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN
878,Dicotyledones,CARYOPHYLLACEAE,Cerastium alpinum,This Plant commonly produced 1 or 2 flowers wh...,NaN,NaN,NaN,NaN


In [15]:
family_counts = out_csv["speciesName"].value_counts()

In [16]:
dict(family_counts)

{'Cerastium alpinum': np.int64(34),
 'Euphorbia paralias L.': np.int64(8),
 'Euphorbia platyphylllos L.': np.int64(8),
 'Veronica spicata': np.int64(7),
 'Cardamine hirsuta L.': np.int64(6),
 'Artemisia maritima': np.int64(6),
 'Poa nemoralis L.': np.int64(5),
 'Atriplex hastata L.': np.int64(5),
 'Plantago maritima': np.int64(5),
 'Salix aurita L.': np.int64(5),
 'Vicia sativa': np.int64(5),
 'Salix arbuscula L.': np.int64(5),
 'Pyrola minor': np.int64(4),
 'Viola lutea': np.int64(4),
 'Erysimum cheiranthoides L.': np.int64(4),
 'Thalictrum minus L. subsp. minus': np.int64(4),
 'Vicia hirsuta': np.int64(4),
 'Taraxacum officinale Weber, s.l.': np.int64(4),
 'Triplemospermum maritimum (L.) Koch': np.int64(4),
 'Crepis foetida': np.int64(4),
 'Poa subcaerulea Sm.': np.int64(4),
 'Hieracium murorum agg., Flora Scotica ed. 1, 437 (1777)': np.int64(4),
 'Gentianella campestris': np.int64(4),
 'Lactuca saligna': np.int64(4),
 'Atriplex littoralis L.': np.int64(4),
 'Atriplex patula L.': np.

In [34]:
list(out_csv["familyName"])

['ACERACEAE',
 'ACERACEAE',
 'ACERACEAE',
 'AMARANTHACEAE',
 'ARALIACEAE',
 'ARISTOLOCHIACEAE',
 'BALSAMINACEAE',
 'BETULACEAE',
 'BETULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAMPANULACEAE',
 'CAPRIFOLIACEAE',
 'CAPRIFOLIACEAE',
 'CAPRIFOLIACEAE',
 'CAPRIFOLIACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CARYOPHYLLACEAE',
 'CA